In [62]:
import os 
# logging.disable(logging.WARNING)
import numpy as np
import tensorflow as tf
tf.autograph.set_verbosity(1)
import logging
import tensorflow_hub as hub
import tensorflow_datasets as tfds

# Split the training set into 60% and 40% to end up with 15,000 examples
# for training, 10,000 examples for validation and 25,000 examples for testing.
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)


In [63]:
# convert data into embedding vectors using a pre-trained text embedding as the first layer
embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=True)

model = tf.keras.Sequential()
model.add(hub_layer) # pre-trained embedding
model.add(tf.keras.layers.Dense(16, activation='relu')) # fully connected layer with 16 hidden units
model.add(tf.keras.layers.Dense(1)) # fully connected layer with a single output node

# binary cross entropy loss function measures the difference between the ground truth distribution and the predictions
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

2023-05-26 14:08:56.505746: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'sentences' with dtype string and shape [?]
	 [[{{node sentences}}]]
2023-05-26 14:08:56.510501: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder' with dtype string and shape [?]
	 [[{{node Placeholder}}]]


In [25]:
# train for 10 epochs/iterations in mini batches of 512 samples
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=10,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/10


2023-05-26 14:00:23.178085: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype string and shape [1]
	 [[{{node Placeholder/_2}}]]
2023-05-26 14:00:23.178575: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype string and shape [1]
	 [[{{node Placeholder/_1}}]]
2023-05-26 14:00:23.291311: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/StatefulPartitionedCall_grad/StatefulPartitio

30/30 [==============================] - ETA: 0s - loss: 0.6555 - accuracy: 0.5269

2023-05-26 14:00:37.597852: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int64 and shape [1]
	 [[{{node Placeholder/_4}}]]
2023-05-26 14:00:37.598396: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype string and shape [1]
	 [[{{node Placeholder/_2}}]]


30/30 [==============================] - 16s 500ms/step - loss: 0.6555 - accuracy: 0.5269 - val_loss: 0.6114 - val_accuracy: 0.5911
Epoch 2/10
30/30 [==============================] - 15s 488ms/step - loss: 0.5524 - accuracy: 0.6857 - val_loss: 0.5110 - val_accuracy: 0.7204
Epoch 3/10
30/30 [==============================] - 15s 493ms/step - loss: 0.4293 - accuracy: 0.8077 - val_loss: 0.4179 - val_accuracy: 0.8154
Epoch 4/10
30/30 [==============================] - 15s 498ms/step - loss: 0.3198 - accuracy: 0.8767 - val_loss: 0.3549 - val_accuracy: 0.8407
Epoch 5/10
30/30 [==============================] - 15s 508ms/step - loss: 0.2356 - accuracy: 0.9148 - val_loss: 0.3218 - val_accuracy: 0.8588
Epoch 6/10
30/30 [==============================] - 15s 503ms/step - loss: 0.1729 - accuracy: 0.9435 - val_loss: 0.3081 - val_accuracy: 0.8603
Epoch 7/10
30/30 [==============================] - 15s 502ms/step - loss: 0.1243 - accuracy: 0.9649 - val_loss: 0.3059 - val_accuracy: 0.8683
Epoch 8/10

KeyboardInterrupt: 

In [20]:
# evaluate
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

2023-05-26 13:57:22.486463: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype string and shape [1]
	 [[{{node Placeholder/_2}}]]
2023-05-26 13:57:22.486871: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int64 and shape [1]
	 [[{{node Placeholder/_4}}]]


49/49 - 3s - loss: 0.3374 - accuracy: 0.8531 - 3s/epoch - 52ms/step
loss: 0.337
accuracy: 0.853
